# Primeras pruebas para el algoritmo de recomendación

In [1]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from joblib import dump, load
from sklearn.preprocessing import normalize

In [2]:
cestas = pd.read_csv('../data/cestas.csv') 

In [3]:
cestas_su = pd.read_csv('../data/cestas_su.csv') 

In [4]:
productos = pd.read_csv('../data/productos.csv') 

# Pruebas teniendo en cuenta las unidades de los artículos

El modelo de recomendación usa una matriz, ya calculada con TfidfVectorizer

In [5]:
def recomienda_tfid(new_basket):
    # Cargar la matriz TF-IDF y el modelo
    tfidf_matrix = load('../models/tfidf_matrix.joblib')
    tfidf = load('../models/tfidf_model.joblib')

    # Convertir la nueva cesta en formato TF-IDF
    new_basket_str = ' '.join(new_basket)
    new_basket_tfidf = tfidf.transform([new_basket_str])

    # Comparar la nueva cesta con las anteriores
    similarities = cosine_similarity(new_basket_tfidf, tfidf_matrix)

    # Obtener los índices de las cestas más similares
    similar_indices = similarities.argsort()[0][-3:]  # Las 3 más similares

    # Crear un diccionario para contar las recomendaciones
    recommendations_count = {}
    total_similarity = 0

    # Recomendar productos de cestas similares
    for idx in similar_indices:
        sim_score = similarities[0][idx]
        total_similarity += sim_score
        products = cestas.iloc[idx]['Cestas'].split()
        # Usar un conjunto para evitar contar productos múltiples veces en la misma cesta
        unique_products = set(products)  # Usar un conjunto para obtener productos únicos
        # Con esto evitamos que la importancia crezca por las unidades
        for product in unique_products:
            if product.strip() not in new_basket:  # Evitar recomendar lo que ya está en la cesta
                recommendations_count[product.strip()] = recommendations_count.get(product.strip(), 0) + sim_score

    # Calcular la probabilidad relativa de cada producto recomendado
    recommendations_with_prob = []
    if total_similarity > 0:  # Verificar que total_similarity no sea cero
        recommendations_with_prob = [(product, score / total_similarity) for product, score in recommendations_count.items()]
    else:
        print("No se encontraron similitudes suficientes para calcular probabilidades.")

    recommendations_with_prob.sort(key=lambda x: x[1], reverse=True)  # Ordenar por puntuación

    # Crear un nuevo DataFrame para almacenar las recomendaciones
    recommendations_data = []
    
    for product, prob in recommendations_with_prob:
        # Buscar la descripción en el DataFrame de productos
        description = productos.loc[productos['ARTICULO'] == product, 'DESCRIPCION']
        if not description.empty:
            recommendations_data.append({
                'ARTICULO': product,
                'DESCRIPCION': description.values[0],  # Obtener el primer valor encontrado
                'PROBABILIDAD': prob
            })

    recommendations_df = pd.DataFrame(recommendations_data)
    
    return recommendations_df


In [6]:
def recomienda_count(new_basket):
    # Cargar la matriz de conteo y el modelo desde la carpeta 'models'
    count_matrix = load('../models/count_matrix.joblib')
    count_vectorizer = load('../models/count_vectorizer.joblib')

    # Convertir la nueva cesta en formato de conteo
    new_basket_str = ' '.join(new_basket)
    new_basket_count = count_vectorizer.transform([new_basket_str])

    # Comparar la nueva cesta con las anteriores
    similarities = cosine_similarity(new_basket_count, count_matrix)

    # Obtener los índices de las cestas más similares
    similar_indices = similarities.argsort()[0][-3:]  # Las 3 más similares

    # Crear un diccionario para contar las recomendaciones
    recommendations_count = {}
    total_similarity = 0

    # Recomendar productos de cestas similares
    for idx in similar_indices:
        sim_score = similarities[0][idx]
        total_similarity += sim_score
        products = cestas.iloc[idx]['Cestas'].split()
        # Usar un conjunto para evitar contar productos múltiples veces en la misma cesta
        unique_products = set(products)  # Usar un conjunto para obtener productos únicos
        # Con esto evitamos que la importancia crezca por las unidades
        for product in unique_products:
            if product.strip() not in new_basket:  # Evitar recomendar lo que ya está en la cesta
                recommendations_count[product.strip()] = recommendations_count.get(product.strip(), 0) + sim_score

    # Calcular la probabilidad relativa de cada producto recomendado
    recommendations_with_prob = []
    if total_similarity > 0:  # Verificar que total_similarity no sea cero
        recommendations_with_prob = [(product, score / total_similarity) for product, score in recommendations_count.items()]
    else:
        print("No se encontraron similitudes suficientes para calcular probabilidades.")

    recommendations_with_prob.sort(key=lambda x: x[1], reverse=True)  # Ordenar por puntuación

    # Crear un nuevo DataFrame para almacenar las recomendaciones
    recommendations_data = []
    
    for product, prob in recommendations_with_prob:
        # Buscar la descripción en el DataFrame de productos
        description = productos.loc[productos['ARTICULO'] == product, 'DESCRIPCION']
        if not description.empty:
            recommendations_data.append({
                'ARTICULO': product,
                'DESCRIPCION': description.values[0],  # Obtener el primer valor encontrado
                'PROBABILIDAD': prob
            })

    recommendations_df = pd.DataFrame(recommendations_data)
    
    return recommendations_df

In [7]:
def recomienda_tf(new_basket): 
    # Cargar la matriz TF y el modelo
    tf_matrix = load('../models/tf_matrix.joblib')
                      
    count = load('../models/count_vectorizer.joblib')
    # Convertir la nueva cesta en formato TF (Term Frequency)
    new_basket_str = ' '.join(new_basket)
    new_basket_vector = count.transform([new_basket_str])
    new_basket_tf = normalize(new_basket_vector, norm='l1')  # Normalizamos la matriz count de la cesta actual
    # Comparar la nueva cesta con las anteriores
    similarities = cosine_similarity(new_basket_tf, tf_matrix)
    # Obtener los índices de las cestas más similares
    similar_indices = similarities.argsort()[0][-4:]  # Las 4 más similares
    # Crear un diccionario para contar las recomendaciones
    recommendations_count = {}
    total_similarity = 0
    # Recomendar productos de cestas similares
    for idx in similar_indices:
        sim_score = similarities[0][idx]
        total_similarity += sim_score  # Suma de las similitudes
        products = cestas.iloc[idx]['Cestas'].split()
        # Usar un conjunto para evitar contar productos múltiples veces en la misma cesta
        unique_products = set(products)  # Usar un conjunto para obtener productos únicos
        # Con esto evitamos que la importancia crezca por las unidades
        for product in unique_products:
            if product.strip() not in new_basket:  # Evitar recomendar lo que ya está en la cesta
                recommendations_count[product.strip()] = recommendations_count.get(product.strip(), 0) + sim_score
                # Almacena el conteo de la relevancia de cada producto basado en cuántas veces aparece en las cestas similares, ponderado por la similitud de cada cesta.
    # Calcular la probabilidad relativa de cada producto recomendado
    recommendations_with_prob = []
    if total_similarity > 0:  # Verificar que total_similarity no sea cero
        recommendations_with_prob = [(product, score / total_similarity) for product, score in recommendations_count.items()]
    else:
        print("No se encontraron similitudes suficientes para calcular probabilidades.")
     
    recommendations_with_prob.sort(key=lambda x: x[1], reverse=True)  # Ordenar por puntuación
    # Crear un nuevo DataFrame para almacenar las recomendaciones
    recommendations_data = []
    
    for product, score in recommendations_with_prob:
        # Buscar la descripción en el DataFrame de productos
        description = productos.loc[productos['ARTICULO'] == product, 'DESCRIPCION']
        if not description.empty:
            recommendations_data.append({
                'ARTICULO': product,
                'DESCRIPCION': description.values[0],  # Obtener el primer valor encontrado
                'RELEVANCIA': score
            })
    recommendations_df = pd.DataFrame(recommendations_data)
    
    return recommendations_df

# Pruebas

In [8]:
# Ejemplo de una nueva cesta para recomendar productos.
# Cesta de prueba
new_basket = ['08049011087', '08049011087','0803B1103000', '0803B1103000']
new_basket_2 = ['00R1BTGCBL5','00R1BTGBL9']
basket = ['00R1BTGCBL5','00R1BTGBL9']
new_basket_3 = ['0113LU01', '0113LU12']


# Prueba con TFID

In [9]:
recomendacion_tfid = recomienda_tfid(new_basket)
recomendacion_tfid

,ARTICULO,DESCRIPCION,PROBABILIDAD
0,0605DG,BOTE DE DECAPANTE CON PINCEL 100 GRAMOS PARA S...,0.343075
1,0719VCD3048CS1,PANEL FIJO DE DUCHA Y BARRA SOPORTE NIDIA 3 DE...,0.343075
2,0201FC6328,F 63 MANGUITO ( ENLACE ) PARA COBRE COMPRESIÓN...,0.343075
3,013823401,MONOMANDO DE BAÑO DUCHA ENKEL CROMADO REF. 234...,0.343075
4,0660047,GRIFO FLOTADOR CON BOYA PARA CISTERNA DE WC AL...,0.343075
5,06076,ROLLO ESTAÑO PLATA TIPO 6%,0.343075
6,013823402,MONOMANDO DE DUCHA ENKEL CROMADO REF. 23402 CI...,0.343075
7,08041311087,TE DE PVC DE 110 87º MACHO HEMBRA GRIS,0.342163
8,0804241C4032,CASQUILLO REDUCTOR DE PVC DE 40 32 GRIS,0.314762
9,05TE11AN0650,TERMO ELECTRICO MODELO A06 ANTICALC DE 50Litro...,0.314762


# Prueba con COUNT

In [10]:
recomendacion_count = recomienda_count(new_basket)
recomendacion_count

,ARTICULO,DESCRIPCION,PROBABILIDAD
0,0605DG,BOTE DE DECAPANTE CON PINCEL 100 GRAMOS PARA S...,0.338576
1,0719VCD3048CS1,PANEL FIJO DE DUCHA Y BARRA SOPORTE NIDIA 3 DE...,0.338576
2,0201FC6328,F 63 MANGUITO ( ENLACE ) PARA COBRE COMPRESIÓN...,0.338576
3,013823401,MONOMANDO DE BAÑO DUCHA ENKEL CROMADO REF. 234...,0.338576
4,0660047,GRIFO FLOTADOR CON BOYA PARA CISTERNA DE WC AL...,0.338576
5,06076,ROLLO ESTAÑO PLATA TIPO 6%,0.338576
6,013823402,MONOMANDO DE DUCHA ENKEL CROMADO REF. 23402 CI...,0.338576
7,08041311087,TE DE PVC DE 110 87º MACHO HEMBRA GRIS,0.336209
8,0804241C4032,CASQUILLO REDUCTOR DE PVC DE 40 32 GRIS,0.325215
9,05TE11AN0650,TERMO ELECTRICO MODELO A06 ANTICALC DE 50Litro...,0.325215


# Prueba Tf + Normalize

In [11]:
recomendacion_tf = recomienda_tf(new_basket_3)
recomendacion_tf

,ARTICULO,DESCRIPCION,RELEVANCIA
0,0113LU31,MONOMANDO DE DUCHA LUCA CROMADO FELIU BOET 14424,0.521353
1,0770BR0600CS,BARRA RECTA DE 600 mm INOX SAT 73233NP MEDICLI...,0.265506
2,0811500P,BOTE DE COLA N-30 DE 1/2 KILO CON PINCEL PARA ...,0.255848
3,0804135045,TE DE PVC DE 50 45º HEMBRA HEMBRA GRIS,0.255848


## Conclusión sobre las pruebas
1. Count Vectorizer: Esta función tranforma nuestro histórico de cestas en una matriz donde cada fila
es una compra y en cada columna los diferentes arículos. Los valores serían la cuenta del número de unidades
de cada producto en cada cesta. La desventaja de esta manera de vectorizar es que, como los valores
no están normalizados, a la hora de calcular distancias tendríamos vectores descompensados, pues habrá cestas
de mayor y menor longitud y se podría estar dando más importancia a ciertas cestas por el hecho de ser más extensas.

2. TFidf: Este método si nos devuelve una matriz normalizada, teniendo
en cuenta la frecuencia de aparición de los artículos pero de manera inversa. 
Esto es útil para algoritmos de recomendación como el aplicado en el proyecto de las películas,
ya que de esa manera las palabras más repetidas que serían verbos genéricos, conjunciones, artículos
etc tendrían un peso mucho menor. Pero en este caso, el hecho de que un artículo aparezca en muchas
cestas si que es relevante porque significa que es un artículo con una probabilidad de venta mayor.

Método Final: Count Vectorizer + Normalización l1. 
'l1' normaliza por la suma de la fila. Normalizamos cada fila de la matriz de conteo para que cada 
valor en la matriz represente la frecuencia relativa del término en ese documento.

El análisis se basa en el sentido de los cálculos en cada algoritmo ya que en cuanto a resultados, obtenemos 
recomendaciones muy similres, con la única diferencia en los score que difieren ligeramente.

# Pruebas sin unidades

In [12]:
def recomienda_tf_su(new_basket): 
    # Cargar la matriz TF y el modelo
    tf_matrix_su = load('../models/count_matrix_su.joblib')                
    count = load('../models/count_vectorizer.joblib')

    # Convertir la nueva cesta en formato TF (Term Frequency)
    new_basket_str = ' '.join(new_basket)
    new_basket_vector = count.transform([new_basket_str])
    new_basket_tf = normalize(new_basket_vector, norm='l1')  # Normalizamos la matriz count de la cesta actual
    # Comparar la nueva cesta con las anteriores
    similarities = cosine_similarity(new_basket_tf, tf_matrix_su)
    # Obtener los índices de las cestas más similares
    similar_indices = similarities.argsort()[0][-4:]  # Las 4 más similares
    # Crear un diccionario para contar las recomendaciones
    recommendations_count = {}
    total_similarity = 0
    # Recomendar productos de cestas similares
    for idx in similar_indices:
        sim_score = similarities[0][idx]
        total_similarity += sim_score  # Suma de las similitudes
        products = cestas_su.iloc[idx]['Cestas'].split()
        # Usar un conjunto para evitar contar productos múltiples veces en la misma cesta
        unique_products = set(products)  # Usar un conjunto para obtener productos únicos
        # Con esto evitamos que la importancia crezca por las unidades
        for product in unique_products:
            if product.strip() not in new_basket:  # Evitar recomendar lo que ya está en la cesta
                recommendations_count[product.strip()] = recommendations_count.get(product.strip(), 0) + sim_score
                # Almacena el conteo de la relevancia de cada producto basado en cuántas veces aparece en las cestas similares, ponderado por la similitud de cada cesta.
    # Calcular la probabilidad relativa de cada producto recomendado
    recommendations_with_prob = []
    if total_similarity > 0:  # Verificar que total_similarity no sea cero
        recommendations_with_prob = [(product, score / total_similarity) for product, score in recommendations_count.items()]
    else:
        print("No se encontraron similitudes suficientes para calcular probabilidades.")
     
    recommendations_with_prob.sort(key=lambda x: x[1], reverse=True)  # Ordenar por puntuación
    # Crear un nuevo DataFrame para almacenar las recomendaciones
    recommendations_data = []
    
    for product, score in recommendations_with_prob:
        # Buscar la descripción en el DataFrame de productos
        description = productos.loc[productos['ARTICULO'] == product, 'DESCRIPCION']
        if not description.empty:
            recommendations_data.append({
                'ARTICULO': product,
                'DESCRIPCION': description.values[0],  # Obtener el primer valor encontrado
                'RELEVANCIA': score
            })
    recommendations_df = pd.DataFrame(recommendations_data)
    
    return recommendations_df

In [14]:
recomendacion_tf_su = recomienda_tf_su(basket)
recomendacion_tf_su

,ARTICULO,DESCRIPCION,RELEVANCIA
0,0209611,K 61 RACOR ENLACE MIXTO MACHO DE METAL DE 32 1...,0.259875
1,02521008773,"Uponor Q&E codo hembra 20-Rp1/2""FT",0.259875
2,02521038355,Metros de tubería Uponor Aqua Pipe natural PN6...,0.259875
3,02521008680,Uponor Q&E codo PPSU 20x20,0.259875
4,02521042834,Uponor Q&E Evolution anillo azul 20,0.259875
5,02521022264,"Uponor Q&E racor hembra 20-Rp1/2""FT",0.259875
6,02521008739,"Uponor Q&E racor macho 20-G1/2""MT",0.259875
7,1601ES1,LLAVE DE PASO ESFERA MODELO TAJO 2000 DE 1 ARCO,0.259875
8,02521008822,"Uponor Smart Aqua codo fijación Q&E 20-Rp1/2""F...",0.259875
9,02521038145,Uponor Smart Aqua placa fijación 210x65,0.259875
